<a href="https://colab.research.google.com/github/DVK-MSU/testing/blob/main/spark_session_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

import numpy as np
import pandas as pd
#Функция для передачи результата sql-запроса в pandas DataFrame.
#Напиши select = """ SELECT * FROM table""" и вызови функцию sql(select)
def sql(select):
    SQLdf = spark.sql(select)
    return SQLdf.toPandas()

import pyspark.sql.functions as F

from google.colab import data_table
data_table.disable_dataframe_formatter()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

# Load data

In [ ]:
def reheadering_ETL():
    #extraction data from csv
    path = "/content/drive/MyDrive/Spark_training/california_housing_train.csv"
    sc = spark.read.csv(path)
    sc.createOrReplaceTempView("df")
    #transformation data (reheadering columns)
    sc_pandas = sc.toPandas()
    columns_name = dict(zip(sc_pandas.columns.to_list(),sc_pandas.iloc[0].to_list()))
    sc_pandas.rename(columns=columns_name, inplace=True)
    sc_pandas.drop([0], inplace=True)
    sc = spark.createDataFrame(sc_pandas)
    #loading data to json
    sc.write.json('/content/drive/MyDrive/Spark_training/output', mode='overwrite')
reheadering_ETL()


In [ ]:
spark.read.json('/content/drive/MyDrive/Spark_training/output').show() # output data

+-----------+------------------+---------+-----------+------------------+-------------+-----------+--------------+-----------+
| households|housing_median_age| latitude|  longitude|median_house_value|median_income| population|total_bedrooms|total_rooms|
+-----------+------------------+---------+-----------+------------------+-------------+-----------+--------------+-----------+
| 472.000000|         15.000000|34.190000|-114.310000|      66900.000000|     1.493600|1015.000000|   1283.000000|5612.000000|
| 463.000000|         19.000000|34.400000|-114.470000|      80100.000000|     1.820000|1129.000000|   1901.000000|7650.000000|
| 117.000000|         17.000000|33.690000|-114.560000|      85700.000000|     1.650900| 333.000000|    174.000000| 720.000000|
| 226.000000|         14.000000|33.640000|-114.570000|      73400.000000|     3.191700| 515.000000|    337.000000|1501.000000|
| 262.000000|         20.000000|33.570000|-114.570000|      65500.000000|     1.925000| 624.000000|    326.0000

In [ ]:
sc_renamed = spark.read.json('/content/drive/MyDrive/Spark_training/output')
sc_renamed.dtypes


sucsess


In [ ]:
def FloatType_Transform(sc_renamed):
    #Extract
    sc_renamed = spark.read.json('/content/drive/MyDrive/Spark_training/output')
    #Transform
    from pyspark.sql.functions import col
    return sc_renamed.select([col(col_name).cast('float') for col_name in sc_renamed.columns])
sc_renamed= sc_renamed.transform(FloatType_Transform)
sc_renamed.write.json('/content/drive/MyDrive/Spark_training/output/float_type_transform', mode='overwrite')

In [ ]:
spark.read.json('/content/drive/MyDrive/Spark_training/output/float_type_transform').dtypes

[('households', 'double'),
 ('housing_median_age', 'double'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('median_house_value', 'double'),
 ('median_income', 'double'),
 ('population', 'double'),
 ('total_bedrooms', 'double'),
 ('total_rooms', 'double')]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
